# Imports

In [0]:
import numpy
import pandas
import keras
import tensorflow.contrib.eager as tfe
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.utils import shuffle
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras import Input

Using TensorFlow backend.


# fixing random seed

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# loading dataset

In [0]:
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

# Encoding labels

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [0]:
X.shape

(150, 4)

# Sequential API

## Baseline Model

In [0]:
# define baseline model
def baseline_model():
  # Create model
  model = models.Sequential()
  model.add(layers.Dense(8, activation = 'relu', input_shape = (X.shape[1],)))
  model.add(layers.Dense(3, activation='softmax'))
  # Compile model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [0]:
# KerasClassifier class will be passed on to the fit() function internally used to train the neural network. 
# Debugging is also turned off

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [0]:
# evaluate our model (estimator) on our dataset (X and dummy_y) using a 10-fold cross-validation procedure (kfold).

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [0]:
# evaluate the neural network model on our training data.
# the evaluation of the 10 constructed models for each of the splits of the dataset
# The results are summarized as both the mean and standard deviation of the model accuracy on the dataset

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Baseline: 97.33% (4.42%)


## Smaller Model

In [0]:
# smaller model
def smaller_model():
  # create model
  model = models.Sequential()
  model.add(layers.Dense(2, activation ='relu', input_shape = (X.shape[1],))) # number of features (4) are used as vector in input shape
  model.add(layers.Dense(3, activation ='softmax'))
	# Compile model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

estimator = KerasClassifier(build_fn=smaller_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 76.67% (27.04%)


## Larger Model

In [0]:
# larger model
def larger_model():
  # create model
  model = models.Sequential()
  model.add(layers.Dense(16, activation ='relu', input_shape = (X.shape[1],))) # number of features (4) are used as vector in input shape
  model.add(layers.Dense(3, activation ='softmax'))
	# Compile model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

estimator = KerasClassifier(build_fn=larger_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 94.00% (6.96%)


## Scaled Model

In [0]:
# scaled model
def scaled_model():
  # create model
  model = models.Sequential()
  # number of features (4) are used as vector in input shape
  model.add(layers.Dense(512, activation='relu', input_shape = (X.shape[1],)))
  model.add(layers.Dense(512, activation ='relu'))
  model.add(layers.Dense(3, activation ='softmax'))
	# Compile model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

estimator = KerasClassifier(build_fn=scaled_model, epochs=300, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Scaled: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Scaled: 97.33% (4.42%)


## Tuned Model

In [32]:
# tuned model
def tuned_model():
  # create model
  model = models.Sequential()
  model.add(layers.Dense(8, activation ='relu', input_shape = (X.shape[1],))) # number of features (4) are used as vector in input shape
  model.add(layers.Dense(10, activation='relu'))
  model.add(layers.Dense(10, activation='relu'))
  model.add(layers.Dense(3, activation ='softmax'))
	# Compile model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

estimator = KerasClassifier(build_fn=tuned_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Tuned: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Tuned: 96.67% (4.47%)


# Functional API

## Baseline Model

In [0]:
def baseline_functional():
  input_tensor = Input(shape=(4,))
  x = layers.Dense(8, activation='relu')(input_tensor)
  output_tensor = layers.Dense(3, activation='softmax')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
  # Compile the model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  
  return model

estimator = KerasClassifier(build_fn=baseline_functional, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("baseline_functional: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


baseline_functional: 97.33% (3.27%)


## Smaller Model 

In [0]:
def smaller_functional():
  input_tensor = Input(shape=(4,))
  x = layers.Dense(2, activation='relu')(input_tensor)
  output_tensor = layers.Dense(3, activation='softmax')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
  # Compile the model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  
  return model

estimator = KerasClassifier(build_fn=smaller_functional, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("smaller_functional: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

smaller_functional: 76.67% (27.04%)


## Larger Model

In [0]:
def larger_functional():
  input_tensor = Input(shape=(4,))
  x = layers.Dense(16, activation='relu')(input_tensor)
  output_tensor = layers.Dense(3, activation='softmax')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
  # Compile the model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  
  return model

estimator = KerasClassifier(build_fn=larger_functional, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("larger_functional: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

larger_functional: 96.67% (5.37%)


## Scaled Model

In [0]:
def scaled_functional():
  input_tensor = Input(shape=(4,))
  x = layers.Dense(512, activation='relu')(input_tensor)
  x = layers.Dense(512, activation='relu')(x)
  output_tensor = layers.Dense(3, activation='softmax')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
  # Compile the model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  
  return model

estimator = KerasClassifier(build_fn=scaled_functional, epochs=300, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Scaled_functional: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Scaled_functional: 96.00% (4.42%)


## Tuned Model

In [0]:
def tuned_functional():
  input_tensor = Input(shape=(4,))
  x = layers.Dense(8, activation='relu')(input_tensor)
  x = layers.Dense(10, activation='relu')(x)
  x = layers.Dense(10, activation='relu')(x)
  output_tensor = layers.Dense(3, activation='softmax')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
  # Compile the model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  
  return model

estimator = KerasClassifier(build_fn=tuned_functional, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("tuned_functional: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

tuned_functional: 97.33% (3.27%)


# Model Subclassing

## Baseline Model

In [0]:
class MyBaselineModel(keras.Model):
  
  def __init__(self):
    super(MyBaselineModel,self).__init__('my_baseline')
    self.dense1 = keras.layers.Dense(8, activation='relu')
    self.dense2 = keras.layers.Dense(3, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)
    
def baseline_subclass_model():
  inputs = keras.Input(shape=(4,))
  mymodel = MyBaselineModel()
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

estimator = KerasClassifier(build_fn=baseline_subclass_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("baseline_subclass_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



baseline_subclass_model: 95.33% (5.21%)


## Smaller Model

In [0]:
class MySmallerModel(keras.Model):  # change
  
  def __init__(self):
    super(MySmallerModel,self).__init__('my_smaller')  # change
    self.dense1 = keras.layers.Dense(2, activation='relu')          # change
    self.dense2 = keras.layers.Dense(3, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)      # change
    return self.dense2(x)
    
def smaller_subclass_model():
  inputs = keras.Input(shape=(4,))
  mymodel = MySmallerModel()        # change
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

estimator = KerasClassifier(build_fn=smaller_subclass_model, epochs=200, batch_size=5, verbose=0)    # change

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("smaller_subclass_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))  # change

smaller_subclass_model: 81.33% (29.48%)


## Larger Model

In [0]:
class MyLargerModel(keras.Model):  # change
  
  def __init__(self):
    super(MyLargerModel,self).__init__('my_larger')   # change
    self.dense1 = keras.layers.Dense(16, activation='relu')           # change
    self.dense2 = keras.layers.Dense(3, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)     # change
    return self.dense2(x)
    
def larger_subclass_model():  # change
  inputs = keras.Input(shape=(4,))
  mymodel = MyLargerModel()        # change
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

estimator = KerasClassifier(build_fn=larger_subclass_model, epochs=200, batch_size=5, verbose=0)    # change

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("larger_subclass_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))  # change

larger_subclass_model: 96.00% (5.33%)


## Scaled Model

In [0]:
class MyScaledModel(keras.Model):  # change
  
  def __init__(self):
    super(MyScaledModel,self).__init__('my_scaled')                  # change     
    self.dense1 = keras.layers.Dense(512, activation='relu') 
    self.dense2 = keras.layers.Dense(512, activation='relu')            # change
    self.dense3 = keras.layers.Dense(3, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)       # change 
    return self.dense3(x)
    
def scaled_subclass_model():  # change
  inputs = keras.Input(shape=(4,))
  mymodel = MyScaledModel()        # change
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

estimator = KerasClassifier(build_fn=scaled_subclass_model, epochs=300, batch_size=5, verbose=0)    # change

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("scaled_subclass_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))  # change

scaled_subclass_model: 96.67% (5.37%)


## Tuned Model

In [31]:
class MyTunedModel(keras.Model):  # change
  
  def __init__(self):
    super(MyTunedModel,self).__init__('my_tuned')   # change     
    self.dense1 = keras.layers.Dense(8, activation='relu')
    self.dense2 = keras.layers.Dense(10, activation='relu')   
    self.dense3 = keras.layers.Dense(10, activation='relu')   
    self.dense4 = keras.layers.Dense(3, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)      # change
    x = self.dense3(x)  
    return self.dense4(x)
    
def tuned_subclass_model():  # change
  inputs = keras.Input(shape=(4,))
  mymodel = MyTunedModel()        # change
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

estimator = KerasClassifier(build_fn=tuned_subclass_model, epochs=100, batch_size=5, verbose=0)    # change

kfold = KFold(n_splits=10, shuffle=True, random_state=None)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("tuned_subclass_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))  # change

tuned_subclass_model: 98.67% (2.67%)


# Without Scikit-learn library


## Tuned Model using Custom K-Fold Validation Method (without Scikit-learn Library)

In [0]:
# tuned model
def tuned_custom():
  # create model
  model = models.Sequential()
  model.add(layers.Dense(8, activation ='relu', input_shape = (X.shape[1],))) # number of features (4) are used as vector in input shape
  model.add(layers.Dense(10, activation='relu'))
  model.add(layers.Dense(10, activation='relu'))
  model.add(layers.Dense(3, activation ='softmax'))
	# Compile model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [0]:
# shuffling data
dataframe = shuffle(dataframe)

# checking data
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [0]:
k = 10
num_val_samples = len(X) // 10
num_epochs = 200
all_scores = numpy.array([])

for i in range(k):
  print('processing fold # ', i+1)
  # prepare the validation data: data from partition # k
  val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = dummy_y[i * num_val_samples: (i + 1) * num_val_samples]
  
  # prepare the training data: data from data - k
  partial_train_data = numpy.concatenate(                    
      [X[:i * num_val_samples],
      X[(i + 1 ) * num_val_samples:]],
  axis = 0)
  partial_train_targets = numpy.concatenate(
      [dummy_y[:i * num_val_samples],
      dummy_y[(i + 1 ) * num_val_samples:]],
  axis = 0)
  
  # Build the Keras Models (already commpiled)
  model = tuned_custom()                              # change
  
  # Train the model (in silence mode, verbose = 0)
  history = model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size = 5, verbose = 0)
  
  # Evaluate the model on the validation data
  val_categoical_crossentropy, val_adam = model.evaluate(val_data, val_targets, verbose = 0)
  
  all_scores = numpy.append(all_scores, val_adam)
  print("K-Fold Witout Scikit-learn : (Accuracy)Mean %.2f%% (Error)Std (%.2f%%)" % (all_scores.mean()*100, all_scores.std()*100))
       

processing fold #  1
K-Fold Witout Scikit-learn : (Accuracy)Mean 100.00% (Error)Std (0.00%)
processing fold #  2
K-Fold Witout Scikit-learn : (Accuracy)Mean 100.00% (Error)Std (0.00%)
processing fold #  3
K-Fold Witout Scikit-learn : (Accuracy)Mean 97.78% (Error)Std (3.14%)
processing fold #  4
K-Fold Witout Scikit-learn : (Accuracy)Mean 98.33% (Error)Std (2.89%)
processing fold #  5
K-Fold Witout Scikit-learn : (Accuracy)Mean 97.33% (Error)Std (3.27%)
processing fold #  6
K-Fold Witout Scikit-learn : (Accuracy)Mean 97.78% (Error)Std (3.14%)
processing fold #  7
K-Fold Witout Scikit-learn : (Accuracy)Mean 98.10% (Error)Std (3.01%)
processing fold #  8
K-Fold Witout Scikit-learn : (Accuracy)Mean 97.50% (Error)Std (3.23%)
processing fold #  9
K-Fold Witout Scikit-learn : (Accuracy)Mean 97.78% (Error)Std (3.14%)
processing fold #  10
K-Fold Witout Scikit-learn : (Accuracy)Mean 98.00% (Error)Std (3.06%)
